In [1]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.6 MB/s 
     |████████████████████████████████| 5.5 MB 64.7 MB/s 
     |████████████████████████████████| 212 kB 70.1 MB/s 
     |████████████████████████████████| 115 kB 75.2 MB/s 
     |████████████████████████████████| 182 kB 62.5 MB/s 
     |████████████████████████████████| 127 kB 65.8 MB/s 
     |████████████████████████████████| 7.6 MB 48.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ECE1786 Project/Remy

Mounted at /content/drive
/content/drive/MyDrive/ECE1786 Project/Remy


In [3]:
import transformers
from datasets import Dataset
from transformers import AutoTokenizer
import pandas as pd

In [4]:
model_checkpoint = "distilgpt2"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:

df = pd.read_pickle("/content/drive/MyDrive/ECE1786 Project/Remy/cocktail_dataset.pkl")  
recipes = []
for i in range(len(df.index)):
    recipe = "RECIPE NAME\n" + df.loc[i, "Name"] + " \n\nRECIPE INGREDIENTS\n"
    skip = False
    for ingredient in df.loc[i, "Ingredients"]:
        # remove asterisks
        ingredient.replace('*', '')
        if "750" in ingredient:
            #skip batch drink recipes
            skip = True
        recipe += ingredient + "\n"
    if skip:
        continue
    recipe += "\nRECIPE INSTRUCTIONS\n" 
    for instruction in  df.loc[i, "Instructions"]:
        if instruction.startswith("*"):
            continue
        recipe += instruction + "\n"
    recipes.append(recipe)
tokenizer.pad_token = tokenizer.eos_token
data = tokenizer(recipes, padding='longest')
data["labels"] = data["input_ids"].copy()

dataset = Dataset.from_dict(data)
print(len(dataset))
print(dataset[5])
print(dataset)



292
{'input_ids': [38827, 4061, 36, 36751, 198, 7738, 18531, 220, 198, 198, 38827, 4061, 36, 3268, 10761, 1961, 40, 15365, 198, 17, 24405, 47553, 29219, 198, 16, 14, 17, 25799, 1667, 292, 354, 2879, 300, 2350, 333, 198, 16, 14, 17, 25799, 350, 2797, 304, 14937, 198, 38, 1501, 680, 25, 1667, 292, 354, 2879, 23612, 198, 198, 38827, 4061, 36, 3268, 46126, 11053, 198, 4550, 262, 47553, 29219, 11, 1667, 292, 354, 2879, 300, 2350, 333, 290, 350, 2797, 304, 14937, 656, 257, 17090, 5405, 351, 4771, 290, 11240, 1566, 880, 12, 354, 2967, 13, 198, 1273, 3201, 656, 257, 24554, 5405, 13, 198, 38, 1501, 680, 351, 257, 1667, 292, 354, 2879, 23612, 13, 198, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 

In [17]:
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-recipes",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=30,
    per_device_train_batch_size=4
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilgpt2/snapshots/ee0b29d316a6a55fd15c50c4a97aca80ab4dd66a/config.json
Model config GPT2Config {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
)

In [19]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 292
  Num Epochs = 30
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 2190
  Number of trainable parameters = 81912576


Step,Training Loss
500,0.868900
1000,0.585100
1500,0.513900
2000,0.475100


Saving model checkpoint to distilgpt2-finetuned-recipes/checkpoint-500
Configuration saved in distilgpt2-finetuned-recipes/checkpoint-500/config.json
Model weights saved in distilgpt2-finetuned-recipes/checkpoint-500/pytorch_model.bin
Saving model checkpoint to distilgpt2-finetuned-recipes/checkpoint-1000
Configuration saved in distilgpt2-finetuned-recipes/checkpoint-1000/config.json
Model weights saved in distilgpt2-finetuned-recipes/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to distilgpt2-finetuned-recipes/checkpoint-1500
Configuration saved in distilgpt2-finetuned-recipes/checkpoint-1500/config.json
Model weights saved in distilgpt2-finetuned-recipes/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to distilgpt2-finetuned-recipes/checkpoint-2000
Configuration saved in distilgpt2-finetuned-recipes/checkpoint-2000/config.json
Model weights saved in distilgpt2-finetuned-recipes/checkpoint-2000/pytorch_model.bin


Training completed. Do not forget to share your m

TrainOutput(global_step=2190, training_loss=0.5982914545764662, metrics={'train_runtime': 566.1447, 'train_samples_per_second': 15.473, 'train_steps_per_second': 3.868, 'total_flos': 713064542699520.0, 'train_loss': 0.5982914545764662, 'epoch': 30.0})

In [34]:
prompt = "RECIPE NAME\n Vanilla"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda:0")

outputs = model.generate(input_ids, do_sample=True, max_length=150,temperature = 0.7, top_p = 1, return_dict_in_generate=True, output_scores=True)
print(tokenizer.batch_decode(outputs.sequences, skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


RECIPE NAME
 Vanilla 

RECIPE INGREDIENTS
3-4 ounces coffee liqueur
1/2 ounce lemon juice, freshly squeezed
1/4 ounce simple syrup
Garnish: lemon twist

RECIPE INSTRUCTIONS
Add the coffee liqueur into a shot glass with ice and shake until well-chilled.
Strain into a rocks glass over fresh ice.

